In [61]:
from enum import Enum
import json 

# import imgviz 
import numpy as np

from __future__ import print_function

# import argparse
import glob
import os

import imgviz

import labelme
from pathlib import Path 
import pandas as pd

import os.path as osp

import numpy as np
import PIL.Image


import math
import uuid

# import numpy as np
# import PIL.Image
import PIL.ImageDraw

from labelme.logger import logger
# cmap_a=imgviz.label_colormap()
# cmap_a


class Abstract(object):
    pass

# mask color codes
class MaskColorMap(Enum):
    Unlabelled = (155, 155, 155)
    Building = (60, 16, 152)
    Land = (132, 41, 246)
    Road = (110, 193, 228)
    Vegetation = (254, 221, 58)
    Water = (226, 169, 41)


Unlabelled_hex='#%02x%02x%02x' % (155, 155, 155)
Building_hex='#%02x%02x%02x' % (60, 16, 152)
Land_hex='#%02x%02x%02x' % (132, 41, 246)
Road_hex='#%02x%02x%02x' % (110, 193, 228)
Vegetation_hex='#%02x%02x%02x' % (254, 221, 58)
Water_hex='#%02x%02x%02x' % (226, 169, 41)

list_hex_aa=[Unlabelled_hex,  Building_hex, Land_hex, Road_hex, Vegetation_hex, Water_hex]
list_hex_aa

['#9b9b9b', '#3c1098', '#8429f6', '#6ec1e4', '#fedd3a', '#e2a929']

In [59]:
# def lblsave(filename, lbl, colormap):

#     if osp.splitext(filename)[1] != ".png":
#         filename += ".png"
#     # Assume label ranses [-1, 254] for int32,
#     # and [0, 255] for uint8 as VOC.
#     if lbl.min() >= -1 and lbl.max() < 255:
#         lbl_pil = PIL.Image.fromarray(lbl.astype(np.uint8), mode="P")
        
#         # colormap = imgviz.label_colormap()
#         # # changed 
#         # # colormap[1]=np.array([222,89,127], dtype=np.uint8)
#         # colormap[0]=np.array([155, 155, 155], dtype=np.uint8)
#         # colormap[1]=np.array([110, 193, 228], dtype=np.uint8)
#         # colormap[2]=np.array([60, 16, 152], dtype=np.uint8)
        
#         lbl_pil.putpalette(colormap.flatten())
#         lbl_pil.save(filename)
#     else:
#         raise ValueError(
#             "[%s] Cannot save the pixel-wise class label as PNG. "
#             "Please consider using the .npy format." % filename
#         )

In [62]:
def lblsave(filename, lbl, colormap):

    if osp.splitext(filename)[1] != ".png":
        filename += ".png"
    # Assume label ranses [-1, 254] for int32,
    # and [0, 255] for uint8 as VOC.
    if lbl.min() >= -1 and lbl.max() < 255:
        lbl_pil = PIL.Image.fromarray(lbl.astype(np.uint8), mode="P")
        
        # colormap = imgviz.label_colormap()
        # # changed 
        # # colormap[1]=np.array([222,89,127], dtype=np.uint8)
        # colormap[0]=np.array([155, 155, 155], dtype=np.uint8)
        # colormap[1]=np.array([110, 193, 228], dtype=np.uint8)
        # colormap[2]=np.array([60, 16, 152], dtype=np.uint8)
        
        lbl_pil.putpalette(colormap.flatten())
        lbl_pil.save(filename)
    else:
        raise ValueError(
            "[%s] Cannot save the pixel-wise class label as PNG. "
            "Please consider using the .npy format." % filename
        )

def shape_to_mask(
    img_shape, points, shape_type=None, 
    line_width=10, point_size=5, 
    # fillcolor_polygon='white' # change 
):
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    draw = PIL.ImageDraw.Draw(mask)
    xy = [tuple(point) for point in points]

    print('shape type=', shape_type, ' | ')
    if shape_type == "circle":
        assert len(xy) == 2, "Shape of shape_type=circle must have 2 points"
        (cx, cy), (px, py) = xy
        d = math.sqrt((cx - px) ** 2 + (cy - py) ** 2)
        draw.ellipse([cx - d, cy - d, cx + d, cy + d], outline=1, fill=1)
    elif shape_type == "rectangle":
        assert len(xy) == 2, "Shape of shape_type=rectangle must have 2 points"
        draw.rectangle(xy, outline=1, fill=1)
    elif shape_type == "line":
        assert len(xy) == 2, "Shape of shape_type=line must have 2 points"
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "linestrip":
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "point":
        assert len(xy) == 1, "Shape of shape_type=point must have 1 points"
        cx, cy = xy[0]
        r = point_size
        draw.ellipse([cx - r, cy - r, cx + r, cy + r], outline=1, fill=1)
    else:
        assert len(xy) > 2, "Polygon must have points more than 2"
        draw.polygon(xy=xy, outline=1, fill=1)
        #draw.polygon(xy=xy, outline=1, fill=fillcolor_polygon)
    mask = np.array(mask, dtype=bool)
    return mask


def shapes_to_label(img_shape, shapes, label_name_to_value):
    cls = np.zeros(img_shape[:2], dtype=np.int32)
    ins = np.zeros_like(cls)
    instances = []
    for shape in shapes:
        points = shape["points"]
        label = shape["label"]
        group_id = shape.get("group_id")
        if group_id is None:
            group_id = uuid.uuid1()
        shape_type = shape.get("shape_type", None)

        cls_name = label
        instance = (cls_name, group_id)

        if instance not in instances:
            instances.append(instance)
        ins_id = instances.index(instance) + 1
        cls_id = label_name_to_value[cls_name]

        mask = shape_to_mask(img_shape[:2], points, shape_type)
        cls[mask] = cls_id
        ins[mask] = ins_id

    return cls, ins



In [63]:
def main(json_dir, colormap, list_labels):
    args=Abstract()
    args.input_dir=str( Path(json_dir) )
    args.output_dir=str( Path(json_dir).joinpath('output').joinpath(  pd.Timestamp.now().strftime('%Y%m%dT%H%M%S')  ) )
    args.noviz=False 
    # args.labels=str( Path(json_dir).joinpath('labels.txt')  )

    os.makedirs(args.output_dir, exist_ok=True)
    os.makedirs(osp.join(args.output_dir, "JPEGImages"), exist_ok=True)
    os.makedirs(osp.join(args.output_dir, "SegmentationClass"), exist_ok=True)
    os.makedirs(osp.join(args.output_dir, "SegmentationClassPNG"), exist_ok=True)
    if not args.noviz:
        os.makedirs(
            osp.join(args.output_dir, "SegmentationClassVisualization"), exist_ok=True 
        )
    print("Creating dataset:", args.output_dir)

    class_names = []
    class_name_to_id = {}
    
    for i, line in enumerate(list_labels):
        class_id = i - 1  # starts with -1
        class_name = line.strip()
        class_name_to_id[class_name] = class_id
        if class_id == -1:
            #assert class_name == "__ignore__"
            class_name='__ignore__';
            assert class_name == "__ignore__"
            continue
        elif class_id == 0:
            assert class_name == "_background_"
        class_names.append(class_name)
        print('class id=', class_id, ' | class name=', class_name )

    class_names = tuple(class_names)
    print("class_names:", class_names)
    out_class_names_file = osp.join(args.output_dir, "class_names.txt")
    with open(out_class_names_file, "w") as f:
        f.writelines("\n".join(class_names))
    print("Saved class_names:", out_class_names_file)

    for filename in glob.glob(osp.join(args.input_dir, "*.json")):
        print("Generating dataset from:", filename)

        label_file = labelme.LabelFile(filename=filename)

        base = osp.splitext(osp.basename(filename))[0]
        out_img_file = osp.join(args.output_dir, "JPEGImages", base + ".jpg")
        out_lbl_file = osp.join(
            args.output_dir, "SegmentationClass", base + ".npy"
        )
        out_png_file = osp.join(
            args.output_dir, "SegmentationClassPNG", base + ".png"
        )
        if not args.noviz:
            out_viz_file = osp.join(
                args.output_dir,
                "SegmentationClassVisualization",
                base + ".jpg",
            )

        with open(out_img_file, "wb") as f:
            f.write(label_file.imageData)
        img = labelme.utils.img_data_to_arr(label_file.imageData)

        lbl, _ = shapes_to_label(
            img_shape=img.shape,
            shapes=label_file.shapes,
            label_name_to_value=class_name_to_id,
        )

        lblsave(out_png_file, lbl, colormap=colormap)

        np.save(out_lbl_file, lbl)

        if not args.noviz:
            viz = imgviz.label2rgb(
                lbl,
                imgviz.rgb2gray(img),
                font_size=15,
                label_names=class_names,
                loc="rb",
            )
            imgviz.io.imsave(out_viz_file, viz)


In [66]:
json_dir="/Users/lunal.brown/Personal/DataScience/github/multimedia/data/UCMerced/dom09_dataset"

list( MaskColorMap.Unlabelled.value )

colormap = imgviz.label_colormap()
colormap[0]=np.array(list( MaskColorMap.Unlabelled.value ), dtype=np.uint8)
colormap[1]=np.array(list( MaskColorMap.Water.value ), dtype=np.uint8)
colormap[2]=np.array(list( MaskColorMap.Land.value), dtype=np.uint8)

list_labels=['ign', '_background_', 'Water', 'Land'] # 0,1はそれぞれ __ignore__, _background_で始まる必要がある。
main(json_dir=json_dir, colormap=colormap, list_labels=list_labels)

Creating dataset: /Users/lunal.brown/Personal/DataScience/github/multimedia/data/UCMerced/dom09_dataset/output/20230507T154453
class id= 0  | class name= _background_
class id= 1  | class name= Water
class id= 2  | class name= Land
class_names: ('_background_', 'Water', 'Land')
Saved class_names: /Users/lunal.brown/Personal/DataScience/github/multimedia/data/UCMerced/dom09_dataset/output/20230507T154453/class_names.txt
Generating dataset from: /Users/lunal.brown/Personal/DataScience/github/multimedia/data/UCMerced/dom09_dataset/beach39.json
shape type= polygon  | 
shape type= polygon  | 


In [ ]:
"#50E3C2", "geometry_config": {}}, 
    {"title": "Land (unpaved area)", "shape": "polygon", "color": "#F5A623", "geometry_config": {}}, 
    {"title": "Road", "shape": "polygon", "color": "#DE597F", "geometry_config": {}}, 
    {"title": "Building", "shape": "polygon", "color": "#D0021B", "geometry_config": {}}, 
    {"title": "Vegetation", "shape": "polygon", "color": "#417505", "geometry_config": {}}, 
    {"title": "Unlabeled", "shape": "polygon", "color": "#9B9B9B",

In [30]:
hcolor_a=["#F5A623", "#DE597F","#D0021B","#417505","#9B9B9B"] 

In [29]:
# import pandas as pd
def apply_formatting(col, hex_colors):
    for hex_color in hex_colors:
        if col.name == hex_color:
            return [f'background-color: {hex_color}' for c in col.values]
                
def display_hex_colors(hex_colors: list ):
    df = pd.DataFrame(hex_colors).T
    df.columns = hex_colors
    df.iloc[0,0:len(hex_colors)] = ""
    display(df.style.apply(lambda x: apply_formatting(x, hex_colors)))

In [31]:
display_hex_colors(hcolor_a)

,#F5A623,#DE597F,#D0021B,#417505,#9B9B9B
0,,,,,


In [41]:
display_hex_colors(list_hex_aa)

,#9b9b9b,#3c1098,#8429f6,#6ec1e4,#fedd3a,#e2a929
0,,,,,,


In [ ]:
colormap = imgviz.label_colormap()
# changed 
# colormap[1]=np.array([222,89,127], dtype=np.uint8)
colormap[0]=np.array([155, 155, 155], dtype=np.uint8)
colormap[1]=np.array([110, 193, 228], dtype=np.uint8)
colormap[2]=np.array([60, 16, 152], dtype=np.uint8)

In [22]:
json_path="/Users/lunal.brown/Personal/DataScience/github/multimedia/data/UCMerced/dom09_dataset/beach39.json"
ins_json=Abstract()
# ins_json.df=pd.read_json(json_path, orient='records')
# ins_json.df
with open(json_path, 'r') as fi:
    ins_json.data=json.load(fi)

In [28]:
ins_json.data['shapes'][1]

{'label': 'Land',
 'points': [[1.249299719887965, 155.7310924369748],
  [68.47619047619048, 108.11204481792718],
  [104.89075630252101, 90.74509803921569],
  [145.50700280112045, 80.3809523809524],
  [164.5546218487395, 68.61624649859945],
  [205.17086834733894, 60.49299719887956],
  [229.82072829131653, 54.050420168067234],
  [252.50980392156862, 55.7310924369748],
  [255.0, 56.04236694677872],
  [255.0, 0.5490196078431475],
  [0.0, 0.5490196078431475]],
 'group_id': None,
 'description': '',
 'shape_type': 'polygon',
 'flags': {}}